In [31]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [32]:
DATA_DIR = '/content/gdrive/MyDrive/QC-BDMLI-Assessment/Dirty Mark Classifier/Images'

In [33]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [34]:
# Define paths
train_critical_dir = os.path.join(DATA_DIR, 'Critical')
train_major_dir = os.path.join(DATA_DIR, 'Major')
train_minor_dir = os.path.join(DATA_DIR, 'Minor')


In [35]:
# Define data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 80% training, 20% validation
)

# Set up generator for training data
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    subset='training',  # specify subset for training data
    classes=['Critical', 'Major', 'Minor']  # specifying classes explicitly
)

# Set up generator for validation data
validation_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    subset='validation',  # specify subset for validation data
    classes=['Critical', 'Major', 'Minor']  # specifying classes explicitly
)


Found 48 images belonging to 4 classes.
Found 12 images belonging to 4 classes.


In [36]:
# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
# Calculate steps_per_epoch
steps_per_epoch = train_generator.samples // train_generator.batch_size
if train_generator.samples % train_generator.batch_size != 0:
    steps_per_epoch += 1

# Calculate validation_steps
validation_steps = validation_generator.samples // validation_generator.batch_size
if validation_generator.samples % validation_generator.batch_size != 0:
    validation_steps += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

Epoch 1/20
3/3 [==============================] - 19s 8s/step - loss: 1.5438 - accuracy: 0.1667 - val_loss: 1.3872 - val_accuracy: 0.2500
Epoch 2/20
3/3 [==============================] - 3s 875ms/step - loss: 1.4043 - accuracy: 0.2292 - val_loss: 1.3859 - val_accuracy: 0.2500
Epoch 3/20
3/3 [==============================] - 4s 931ms/step - loss: 1.3926 - accuracy: 0.1667 - val_loss: 1.3866 - val_accuracy: 0.2500
Epoch 4/20
3/3 [==============================] - 3s 1s/step - loss: 1.3878 - accuracy: 0.2708 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 5/20
3/3 [==============================] - 3s 682ms/step - loss: 1.3873 - accuracy: 0.2083 - val_loss: 1.3865 - val_accuracy: 0.2500
Epoch 6/20
3/3 [==============================] - 3s 995ms/step - loss: 1.3872 - accuracy: 0.1875 - val_loss: 1.3862 - val_accuracy: 0.2500
Epoch 7/20
3/3 [==============================] - 3s 1s/step - loss: 1.3851 - accuracy: 0.3125 - val_loss: 1.3860 - val_accuracy: 0.2500
Epoch 8/20
3/3 [============